In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from dotenv import load_dotenv
load_dotenv(".env")
from src.domain import Track, User, Setlist
from src.driver import SampleDriverImpl
from src.repository import SampleRepository
from src.solver import QuboSolver
from IPython.display import Image, HTML

## 汎用関数の定義

In [2]:
def millsec_to_str(millSeconds: int, hour: bool = False):
    m, s = divmod(millSeconds / 1000, 60)
    h, m = divmod(m, 60)
    if hour:
        str_duration = "%d:%02d:%02d" % (h, m, s)
    else:
        str_duration = "%02d:%02d" % (m, s)
    return str_duration

def url_to_img(src: str):
    return f'<img src="{src}" />'

def display_setlist(setlist: Setlist):
    time_df = pd.DataFrame([[millsec_to_str(setlist.total_time, hour=True)]], columns=["合計"], index=["時間"])
     
    preference_df = pd.DataFrame(
       [setlist.scores + [setlist.score_sum, setlist.score_avg, setlist.score_var]],
        columns=[[f"ユーザー{i+1}" for i in range(len(setlist.scores))] + ['合計', '平均', '分散']],
        index=['好み']
    )
    
    tracks_df = pd.json_normalize(map(
        lambda t: dict(
            {"image": t.small_image, "アーティスト": t.artist, "曲名": t.name, "時間": millsec_to_str(t.duration_ms)},
            **{ f"ユーザー{i+1}": "⭐" * v for i, v in enumerate(t.p) }),
        setlist.tracks
    ))

    display(time_df)
    display(preference_df)
    display(HTML(tracks_df.to_html(escape=False, formatters=dict(image=url_to_img))))

## 楽曲のサンプルデータを準備（事前に用意した200曲）

In [3]:
sample_driver = SampleDriverImpl()
sample_repository = SampleRepository(driver=sample_driver)
sample_tracks = sample_repository.get_tracks("sample")

In [4]:
sample_tracks_df = pd.json_normalize(map(
    lambda t: {"ID": t.id, "アーティスト": t.artist, "曲名": t.name, "時間[ms]": millsec_to_str(t.duration_ms)},
    sample_tracks
))
display(sample_tracks_df)

,ID,アーティスト,曲名,時間[ms]
0,JPB600057201,浜崎あゆみ,M,04:26
1,JPB601900504,DA PUMP,桜,03:26
2,JPB601902111,DA PUMP,P.A.R.T.Y. 〜ユニバース・フェスティバル〜,03:32
3,JPB602002630,平井 大,"Stand by me, Stand by you.",03:13
4,JPB602051826,マカロニえんぴつ,恋人ごっこ,03:20
...,...,...,...,...
195,TCJPI1811812,川崎 鷹也,魔法の絨毯,03:32
196,TCJPI1992829,瑛人,香水,04:15
197,TCJPK2087568,yama,春を告げる,03:16
198,TCJPL2098599,Tani Yuuki,Myra,04:15


## 参加者の好みを設定したデータセットを用意

In [5]:
M = 5       # 参加者数
n = 100     # 各参加者のプレイリストの曲数
T = 1800    # 設定時間[秒]
users =[]   # 参加者リスト

for idx in range(M):
    tracks = random.sample(sample_tracks, n)  # ランダムにn個の曲を選択
    tracks = [Track(id=t.id, name=t.name, artist=t.artist, small_image=t.small_image, duration_ms=t.duration_ms, priority=random.randint(1, 3)) for t in random.sample(sample_tracks, n)]
    users.append(User(id=f"user{idx+1}", tracks=tracks))

In [6]:
qubo_solver = QuboSolver(users=users, time_limit=T)

In [7]:
evaluated_tracks_df = pd.json_normalize(map(
    lambda t: dict(
        {"ID": t.id, "アーティスト": t.artist, "曲名": t.name, "時間": millsec_to_str(t.duration_ms)},
        **{ f"ユーザー{i+1}": "⭐" * v for i, v in enumerate(t.p) }
    ),
    qubo_solver.candidates,
))
display(evaluated_tracks_df)

,ID,アーティスト,曲名,時間,ユーザー1,ユーザー2,ユーザー3,ユーザー4,ユーザー5
0,JPTF00809111,Mr.Children,花の匂い,05:10,⭐⭐⭐,,,⭐⭐,
1,JPTF09205801,Mr.Children,虹の彼方へ,03:35,⭐,,⭐,,⭐⭐⭐
2,JPTF09809702,Mr.Children,Prism,04:40,⭐⭐⭐,,,⭐,
3,JPU901803007,King Gnu,Prayer X (Acoustic),01:29,⭐⭐,⭐⭐⭐,,,
4,JPTF09501701,Mr.Children,【es】~Theme of es~,05:49,⭐,⭐,⭐,,⭐
...,...,...,...,...,...,...,...,...,...
189,JPU901702278,King Gnu,サマーレイン・ダイバー,04:17,,,,⭐⭐,
190,JPU901702271,King Gnu,Tokyo Rendez-Vous,04:01,,,,⭐,
191,JPU901702276,King Gnu,ロウラヴ,03:32,,,,,⭐
192,JPB600057201,浜崎あゆみ,M,04:26,,,,,⭐⭐⭐


## Amplify Annealing Engine実行

In [8]:
setlist = qubo_solver.solve(timeout=5000)

In [9]:
display_setlist(setlist)

,合計
時間,0:30:03


,ユーザー1,ユーザー2,ユーザー3,ユーザー4,ユーザー5,合計,平均,分散
好み,15.0,17.0,14.0,15.0,16.0,77.0,15.4,1.04


,image,アーティスト,曲名,時間,ユーザー1,ユーザー2,ユーザー3,ユーザー4,ユーザー5
0,,オレンジスパイニクラブ,キンモクセイ,03:51,⭐⭐⭐,⭐⭐,⭐,⭐⭐⭐,⭐⭐
1,,平井 大,"Stand by me, Stand by you.",03:13,⭐⭐⭐,⭐⭐⭐,,⭐⭐⭐,
2,,LiSA,ROCK-mode,04:16,⭐,⭐⭐⭐,⭐⭐,,⭐⭐⭐
3,,"Rin音, RhymeTube",snow jam,03:05,⭐,⭐⭐,⭐⭐,⭐,⭐
4,,ヨルシカ,ただ君に晴れ,03:18,⭐⭐⭐,⭐,⭐⭐⭐,⭐⭐,⭐
5,,milet,inside you,03:41,⭐⭐,⭐⭐⭐,⭐⭐⭐,,⭐⭐⭐
6,,BTS,Stay Gold,04:03,,,⭐⭐⭐,⭐⭐⭐,⭐⭐⭐
7,,NiziU,Make you happy,03:04,,,,⭐⭐⭐,⭐⭐⭐
